In [ ]:
import math

from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library import QFTGate
from qiskit.circuit import ControlledGate
from qiskit_aer import AerSimulator

from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
from qiskit.quantum_info import random_statevector
import cmath


In [ ]:
# System discretization
n = 3 # number of qubits
length = 5 # full length of x domain

dt = 0.005

# Potential/system parameters
m = 1.0
omega = 40.0

N = 2**n # x samples (dimension of hilbert space)
bounds = length / 2
dx = length / N

plength = 2 * cmath.pi / dx
dp = plength / N

alpha = dx
beta = (-bounds + dx / 2) / (alpha * n) #see 10.1119/1.2894532
gamma = m * omega**2 * alpha**2 / 2.0
alphap = dp
gammap = alphap**2 / (2.0 * m)


In [ ]:
# Real time evolution e^iV(x) from a harmonic potential
# n = number of qubits for state (0 = ancillary, 1 to n will be state qubits)
# for gamma and beta, see DOI 10.1119/1.2894532
def harmpot_rte(n, gamma, beta, dt):
    gate = QuantumCircuit(n+1)

    gate.p(-gamma * dt * (beta*n)**2, 0) # global phase shift, beta^2 are in all terms

    for j in range(n):
        for l in range(n):
            if l == j:
                gate.cp(-gamma * dt * (beta * (2**(l+1)) + 2**(l+j)), 0, l+1)
            else:
                gate.cp(-gamma * dt * beta * 2**l, 0, l+1)
                gate.cp(-gamma * dt * beta * 2**j, 0, j+1)
                gate.mcp(-gamma * dt * 2**(l + j), [0, l+1], j+1)

    return gate

In [ ]:
# Real time evolution e^iT(p) for kinetic energy
# n = number of qubits for state (0 = ancillary, 1 to n will be state qubits)
# for gammap, see DOI 10.1119/1.2894532
# gammap = dp^2/2m * dt
def kinetic_rte(n, gammap, dt):
    gate = QuantumCircuit(n+1)

    for j in range(n):
        for l in range(n):
            sign = (-1 if (l == n-1) ^ (j == n-1) else 1)
            exponent = sign * 2**(l+j)
            if l == j:
                gate.cp(-gammap * dt * exponent, 0, l+1)
            else:
                gate.mcp(-gammap * dt * exponent, [0, l+1], j+1)

    return gate

In [ ]:
def harm_rte(n, gamma, beta, gammap, dt):
    rte = QuantumCircuit(n+1)

    # Construct real time evolution circuit
    pot_rte = harmpot_rte(n, gamma, beta, dt)
    ke_rte = kinetic_rte(n, gammap, dt)
    qft = QFTGate(num_qubits=n)
    iqft = qft.inverse()

    rte.append(pot_rte, qargs=range(n+1))
    rte.append(qft, qargs=range(1, n+1))
    rte.append(ke_rte, qargs=range(n+1))
    rte.append(iqft, qargs=range(1, n+1))

    return rte

In [ ]:
init_state = []
width = 0.3
momentum = 10

def itox(i):
    return -length/2 + (i + 0.5)*dx

for i in range(N):
    init_state.append(0)
    init_state.append(cmath.exp((1j*momentum*itox(i))-((itox(i)/width))**2.0))

init_statevector = Statevector(init_state,
                               tuple([2 for i in range(n+1)]))

In [ ]:
from qiskit.circuit.library import UnitaryGate

# ITE DOI:10.1103/PhysRevResearch.4.033121
m0 = 0.5
kappa = (m0 - 1/math.sqrt(2))
kappa = kappa/abs(kappa)
theta0 = kappa * math.acos((m0 + math.sqrt(1 - m0**2))/math.sqrt(2))
s1 = m0 / math.sqrt(1 - m0**2)



runcount = 10000
s2 = math.sqrt(2)

w = Operator([[1/s2, -1j/s2],
             [1/s2, 1j/s2]])
wgate = UnitaryGate(w)

qb = QuantumRegister(n)
ancillary = QuantumRegister(1)
hadamard = ClassicalRegister(1)
args = [ancillary[0]] + [qb[i] for i in range(n)]

circ = QuantumCircuit(ancillary, qb, hadamard)

#circ.initialize(init_statevector, normalize=True)
rte = harm_rte(n, gamma, beta, gammap, dt*s1)
#crte = rte.control(1)

circ.h(ancillary[0])
circ.append(wgate, ancillary)
circ.x(ancillary)
circ.append(rte, args)
circ.x(ancillary)
circ.append(rte.inverse(), args)
circ.rz(-2*theta0, ancillary)
circ.unitary(wgate.inverse(), ancillary)

circ.measure(ancillary, hadamard)


circ.save_statevector()
print(circ.decompose().draw())

#circ.h(ancillary[0])

#for i in range(300):
    #circ.append(rte, qargs=args)
    #circ.save_statevector(label=str(i))

# Transpile for simulator
simulator = AerSimulator()
circ = transpile(circ, simulator)

# Run and get counts
result = simulator.run(circ, shots=runcount).result()

counts = result.get_counts(circ)
plot_histogram(counts, title='Bell-State counts')
# 3.51935513e-01-3.37845366e-02j


In [ ]:
print(result.data(0)['statevector'])


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

def getdata(i):
    if i == 0:
        return result.data(0)['statevector'].data[1::2]
    else:
        return result.data(0)[str(i-1)].data[1::2]


xs = np.linspace((-length+dx)/2,(length-dx)/2,N,endpoint=True)
#ps = np.linspace(0,plength,N,endpoint=False)

fig, ax = plt.subplots()

pspace = ax.plot(xs, getdata(0))[0]
#pspacei = ax.plot(xs, getdata(0)*1j)[0]
#potentialplot = ax.plot(xs, potentials)[0]
ax.set(xlim=[-bounds,bounds])
ax.set(ylim=[-0.5,0.5])

def update(frame):
    data = getdata(frame)
    data_norm = [abs(datum) for datum in data]
    pspace.set_ydata(data_norm)
    #pspacei.set_ydata(getdata(frame)*1j)
    return pspace

ani = animation.FuncAnimation(fig=fig, func=update, frames=50, interval=1)
ani.save('clungus.gif', dpi=80, writer='imagemagick')
plt.show()
